In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
import matplotlib.pyplot as plt
import numpy as np
from fastai.vision.all import *

In [4]:
# Define preprocessing and augmentation for train data
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(), 
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )

])



In [5]:
# Define validation transformation with no augmentation
val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [13]:
import os
from dotenv import load_dotenv
from torchvision import datasets

# Load environment variables from the .env file
load_dotenv() 
# load dataset
PATH = os.getenv("DATASET_PATH")
if PATH is None:
    raise ValueError("DATASET_PATH environment variable not set. Check your .env file.")

dataset = datasets.ImageFolder(PATH, transform=train_transform)

In [15]:
from torch.utils.data import random_split
# split data into train and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])